# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# raw_data is the city data obtain in the WeatherPy section and exported into a .csv.

raw_data = pd.read_csv("../output_data/raw_data.csv")

raw_data


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Date,Wind Speed
0,Busselton,AU,-33.65,115.33,64.00,57,91,1602547055,8.66
1,Malmesbury,ZA,-33.46,18.73,55.99,87,40,1602547056,3.36
2,Chui,UY,-33.70,-53.46,53.82,91,46,1602547056,7.81
3,Cabo San Lucas,MX,22.89,-109.91,82.99,65,90,1602547056,6.93
4,Punta Arenas,CL,-53.15,-70.92,48.20,70,100,1602547056,25.28
...,...,...,...,...,...,...,...,...,...
495,Fort-Shevchenko,KZ,44.51,50.26,60.19,43,23,1602547118,21.97
496,Aykhal,RU,66.00,111.50,12.20,92,46,1602547118,4.47
497,Sobolevo,RU,54.43,31.90,49.95,94,73,1602547118,6.02
498,Tuatapere,NZ,-46.13,167.68,55.00,64,58,1602547118,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store lats & Lngs in Locations & Humidity in a separate location
locations= raw_data[["Latitude", "Longitude"]].astype(float)
humidity= raw_data["Humidity"].astype(float)

# create heat layer
fig=gmaps.figure()

heat_layer= gmaps.heatmap_layer(locations, weights= humidity, dissipating= False, max_intensity=200,
                              point_radius=4)

#Add Layer
fig.add_layer(heat_layer)

#Display the heatmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# new criteria df: 
    # A max temperature lower than 80 degrees but higher than 70 (fahrenheit).
    # Wind speed less than 10 mph.
    # Zero cloudiness.
    # Latitude more than 10
    # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

raw_data

new_criteria = raw_data[(raw_data["Max Temp"] >= 70) & (raw_data["Max Temp"] <= 80) & \
                  (raw_data["Cloudiness"]==0) & (raw_data["Latitude"]>10)].dropna()
new_criteria 

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Date,Wind Speed
41,Mīnāb,IR,27.15,57.08,73.40,11,0,1602547060,4.70
56,Digri,PK,25.16,69.11,78.71,25,0,1602547062,5.03
64,Pāsighāt,IN,28.07,95.33,73.90,72,0,1602547063,2.82
101,Tohāna,IN,29.70,75.90,71.26,34,0,1602547068,5.08
192,Nouadhibou,MR,20.93,-17.03,74.46,73,0,1602547078,13.71
251,Jampur,PK,29.64,70.60,72.91,22,0,1602547086,6.49
292,Bandar-e Lengeh,IR,26.56,54.88,78.17,32,0,1602547091,11.48
339,Bayjī,IQ,34.93,43.49,72.27,22,0,1602547097,7.43
360,Al Bardīyah,LY,31.76,25.09,72.05,58,0,1602547099,13.69
445,Mora,CM,11.05,14.14,79.74,49,0,1602547111,3.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = new_criteria

# add new column "Hotel Name" with no data

hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Hotel Name
41,Mīnāb,IR,27.15,57.08,73.40,11,0,1602547060,4.70,
56,Digri,PK,25.16,69.11,78.71,25,0,1602547062,5.03,
64,Pāsighāt,IN,28.07,95.33,73.90,72,0,1602547063,2.82,
101,Tohāna,IN,29.70,75.90,71.26,34,0,1602547068,5.08,
192,Nouadhibou,MR,20.93,-17.03,74.46,73,0,1602547078,13.71,
251,Jampur,PK,29.64,70.60,72.91,22,0,1602547086,6.49,
292,Bandar-e Lengeh,IR,26.56,54.88,78.17,32,0,1602547091,11.48,
339,Bayjī,IQ,34.93,43.49,72.27,22,0,1602547097,7.43,
360,Al Bardīyah,LY,31.76,25.09,72.05,58,0,1602547099,13.69,
445,Mora,CM,11.05,14.14,79.74,49,0,1602547111,3.09,


In [6]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

# referenced doc:  https://developers.google.com/places/web-service/search
# referenced types for hotel search: https://developers.google.com/places/web-service/supported_types
# location = "The latitude/longitude around which to retrieve place information.This must be specified as latitude,longitude"



# location = hotel_df["Latitude", "Longitude"].values()


for index, row in hotel_df.iterrows():
    
        # set up a parameters dictionary
        params = {
#             "location": location,
            "keyword": "hotel",
            "radius": 5000,
            "type": "lodging",
            "key": g_key
        }
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        lat = row["Latitude"]
        lng = row["Longitude"]
        
        params["location"] = f"{lat}, {lng}"
        
        # API request
        hotel_query= requests.get(base_url, params=params).json()
        
        # request results 
        hotel_info =hotel_query["results"]
        hotel_info
        
        # add the hotel name to the df
        
        try: 
            hotel_df.loc[index, "Hotel Name"] = hotel_info[0]["name"]
        
        except (KeyError, IndexError):
            print("No Hotel Found.")
            
            
hotel_df       


No Hotel Found.


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Hotel Name
41,Mīnāb,IR,27.15,57.08,73.40,11,0,1602547060,4.70,Minab Tourism Hotel
56,Digri,PK,25.16,69.11,78.71,25,0,1602547062,5.03,Sindh Green Hotel
64,Pāsighāt,IN,28.07,95.33,73.90,72,0,1602547063,2.82,OYO 69281 G&t Hotel
101,Tohāna,IN,29.70,75.90,71.26,34,0,1602547068,5.08,Hotel 7 Seas - Hotels in Tohana
192,Nouadhibou,MR,20.93,-17.03,74.46,73,0,1602547078,13.71,Free Zone Hotel Nouadhibou
251,Jampur,PK,29.64,70.60,72.91,22,0,1602547086,6.49,Sajid hotel
292,Bandar-e Lengeh,IR,26.56,54.88,78.17,32,0,1602547091,11.48,Diplomat Hotel
339,Bayjī,IQ,34.93,43.49,72.27,22,0,1602547097,7.43,
360,Al Bardīyah,LY,31.76,25.09,72.05,58,0,1602547099,13.69,Hotel Burdy
445,Mora,CM,11.05,14.14,79.74,49,0,1602547111,3.09,Hotel le Podoko


In [7]:
# adding NaN to the cities with No Hotel Findings

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
print(len(hotel_df))
hotel_df


10


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Hotel Name
41,Mīnāb,IR,27.15,57.08,73.40,11,0,1602547060,4.70,Minab Tourism Hotel
56,Digri,PK,25.16,69.11,78.71,25,0,1602547062,5.03,Sindh Green Hotel
64,Pāsighāt,IN,28.07,95.33,73.90,72,0,1602547063,2.82,OYO 69281 G&t Hotel
101,Tohāna,IN,29.70,75.90,71.26,34,0,1602547068,5.08,Hotel 7 Seas - Hotels in Tohana
192,Nouadhibou,MR,20.93,-17.03,74.46,73,0,1602547078,13.71,Free Zone Hotel Nouadhibou
251,Jampur,PK,29.64,70.60,72.91,22,0,1602547086,6.49,Sajid hotel
292,Bandar-e Lengeh,IR,26.56,54.88,78.17,32,0,1602547091,11.48,Diplomat Hotel
339,Bayjī,IQ,34.93,43.49,72.27,22,0,1602547097,7.43,NaN
360,Al Bardīyah,LY,31.76,25.09,72.05,58,0,1602547099,13.69,Hotel Burdy
445,Mora,CM,11.05,14.14,79.74,49,0,1602547111,3.09,Hotel le Podoko


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

# drop cities/rows from the df with no Hotel Name (w/NaN): to avoid the "hotel marks" from being displayed on cities w/NaN

hotel_df = hotel_df.dropna()
hotel_df


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Latitude", "Longitude"]]


In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

 NOTE: based on my "new criteria", the current hotel dataframe returned 10 cities. However, a Hotel Name was found for only 9 cities. Per the instructions, only marks for hotels should be added to the heatmap; hence the heatmap only consists of 9 marks, which correspond to the 9 Hotel Names found. 